In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t

/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1170: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


_ColormakerRegistry()

In [3]:
import nglview as nv

In [4]:
molsys = m3t.load('/home/diego/1brs.pdb', selection='chainid 0 or chainid 3')

In [5]:
m3t.view(molsys)

NGLWidget()

In [6]:
dists_CAs = m3t.distance(molsys, selection_1="chainid 0 and name CA", selection_2="chainid 3 and name CA")

In [7]:
chain0_CAs = m3t.select(molsys, "chainid 0 and name CA")
chain3_CAs = m3t.select(molsys, "chainid 3 and name CA")

ii=10
jj=20
frame_index=0

print('La distancia entre el átomo {} y el átomo {} es {}'.format(chain0_CAs[ii], chain3_CAs[jj], dists_CAs[frame_index,ii,jj]))

La distancia entre el átomo 74 y el átomo 1022 es 3.475024595596411 nm


In [8]:
dists_CAs = m3t.distance(molsys, selection_1="chainid 0 and name CA", selection_2="chainid 3 and name CA", output_form='dict')

In [9]:
ii=74
jj=1022
frame_index=0

print('La distancia entre el átomo {} y el átomo {} es {}'.format(ii, jj, dists_CAs[ii][jj][frame_index]))

La distancia entre el átomo 74 y el átomo 1022 es 3.475024595596411 nm


In [10]:
m3t.get(molsys, target='atom', indices=[74,1022], index=True, id=True, name=True, element=True)

[[74, 1022], [75, 2745], ['CA', 'CA'], ['C', 'C']]